# HuggingFace Inference Baseline

Training notebook: https://www.kaggle.com/thedrcat/feedback-prize-huggingface-baseline-training

In [1]:
# Config
batch_size = 1
min_tokens = 5
tok_checkpoint = '../input/longformer_v1'
model_checkpoint = '../input/longformer_v1/pytorch_model.bin'

In [2]:
# Load data
import pandas as pd

train = pd.read_csv('../input/feedback-prize-2021train.csv')
train.head(1)

test = pd.read_csv('../input/feedback-prize-2021/sample_submission.csv')
test.head(1)

FileNotFoundError: [Errno 2] No such file or directory: '../input/feedback-prize-2021/train.csv'

In [11]:
# Setup dictionaries
classes = train.discourse_type.unique().tolist()

from collections import defaultdict

tags = defaultdict()
for i, c in enumerate(classes):
    tags[f'B-{c}'] = i
    tags[f'I-{c}'] = i + len(classes)
tags[f'O'] = len(classes) * 2
tags[f'Special'] = -100
l2i = dict(tags)

i2l = defaultdict()
for k, v in l2i.items(): 
    i2l[v] = k
i2l[-100] = 'Special'
i2l = dict(i2l)

In [12]:
# Helper functions
from pathlib import Path

test_path = Path('../input/feedback-prize-2021/test')

def get_test_text(ids):
    with open(test_path/f'{ids}.txt', 'r') as file: data = file.read()
    return data

In [13]:
# Tokenizer
from transformers import AutoTokenizer
    
tokenizer = AutoTokenizer.from_pretrained(tok_checkpoint, add_prefix_space=True)

In [15]:
len(i2l)-1

In [14]:
# Load model
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer
import torch

model = AutoModelForTokenClassification.from_pretrained(tok_checkpoint, num_labels=7)

model.load_state_dict(torch.load(model_checkpoint))
model.eval();

In [ ]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [ ]:
# We'll use trainer with the loaded model to run inference on test set
trainer = Trainer(
    model,
    data_collator=data_collator,
    tokenizer=tokenizer,
)

In [ ]:
# code that will convert our predictions into prediction strings. we'll skip visualization here. 
# this most likely requires some refactoring

def get_class(c):
    if c == 14: return 'Other'
    else: return i2l[c][2:]

def pred2span(pred, example, viz=False, test=False):
    example_id = example['id']
    n_tokens = len(example['input_ids'])
    classes = []
    all_span = []
    for i, c in enumerate(pred.tolist()):
        if i == n_tokens-1:
            break
        if i == 0:
            cur_span = example['offset_mapping'][i]
            classes.append(get_class(c))
        elif i > 0 and (c == pred[i-1] or (c-7) == pred[i-1]):
            cur_span[1] = example['offset_mapping'][i][1]
        else:
            all_span.append(cur_span)
            cur_span = example['offset_mapping'][i]
            classes.append(get_class(c))
    all_span.append(cur_span)
    
    if test: text = get_test_text(example_id)
    else: text = get_raw_text(example_id)
        
    # map token ids to word (whitespace) token ids
    predstrings = []
    for span in all_span:
        span_start = span[0]
        span_end = span[1]
        before = text[:span_start]
        token_start = len(before.split())
        if len(before) == 0: token_start = 0
        elif before[-1] != ' ': token_start -= 1
        num_tkns = len(text[span_start:span_end+1].split())
        tkns = [str(x) for x in range(token_start, token_start+num_tkns)]
        predstring = ' '.join(tkns)
        predstrings.append(predstring)
                    
    rows = []
    for c, span, predstring in zip(classes, all_span, predstrings):
        e = {
            'id': example_id,
            'discourse_type': c,
            'predictionstring': predstring,
            'discourse_start': span[0],
            'discourse_end': span[1],
            'discourse': text[span[0]:span[1]+1]
        }
        rows.append(e)


    df = pd.DataFrame(rows)
    df['length'] = df['discourse'].apply(lambda t: len(t.split()))
    
    # short spans are likely to be false positives, we can choose a min number of tokens based on validation
    df = df[df.length > min_tokens].reset_index(drop=True)

    return df

In [ ]:
# Load test data
import os 

files = os.listdir('../input/feedback-prize-2021/test')
ids = [x.split('.')[0] for x in files]

df_test = pd.DataFrame()
df_test['id'] = ids
df_test['text'] = df_test['id'].apply(get_test_text)
df_test

In [ ]:
from datasets import Dataset

test_ds = Dataset.from_pandas(df_test)
test_ds

In [ ]:
def tokenize_for_test(examples):

    o = tokenizer(examples['text'], truncation=True, return_offsets_mapping=True, max_length=4096)
  
    return o

In [ ]:
tokenized_test = test_ds.map(tokenize_for_test)
tokenized_test

In [ ]:
predictions, _, _ = trainer.predict(tokenized_test)

In [ ]:
import numpy as np

preds = np.argmax(predictions, axis=-1)
predictions.shape, preds.shape

In [ ]:
dfs = []
for i in range(len(tokenized_test)):
    dfs.append(pred2span(preds[i], tokenized_test[i], test=True))

pred_df = pd.concat(dfs, axis=0)
pred_df['class'] = pred_df['discourse_type']

In [ ]:
sub = pred_df[['id', 'class', 'predictionstring']]

In [ ]:
sub.to_csv('submission.csv', index=False)